# AMAL // Mécanismes d'attention
                        notebook n°1 sur 2

Le travail dépend de la base IMDB qui contient des critiques de films. Le but est d'associer une classe (positif, négatif) à ces critiques. Pour cela, le mieux est de prendre en compte les sentiments exprimés dans les phrases isolées ; or le sens est quelque chose de diffus, comment peut-on savoir où regarder, à quels mots les associer ?  

Ce problème est résolu grâce aux mécanismes d'attention. Ils permettent à un modèle d'apprendre à diriger son intérêt vers des points pertinents en même temps qu'il s'assure de la qualité des classifications.  

https://stats.stackexchange.com/questions/421935/what-exactly-are-keys-queries-and-values-in-attention-mechanisms

In [1]:
# //////////////////////////////////////////////////////////////////////////////////////////////// <useful libraries> ////

# Ne pas oublier d'executer dans le shell avant de lancer python :
# source /users/Enseignants/piwowarski/venv/amal/3.7/bin/activate

from tme9models import *
from tme9training import *
from tme9dataset import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch
import torch.nn as nn
%load_ext tensorboard

# /////////////////////////////////////////////////////////////////////////////////////////////// </useful libraries> ////

In [2]:
# ////////////////////////////////////////////////////////////////////////////////////////////////// <quick testing> ////

# Récupération d'un batch seul pour les tests
batch = next(iter(train_loader))

na = NaiveAttention(50,2).to(device)
sa = SimpleAttention(50,2).to(device)
fa = FurtherAttention(50,2).to(device)
la = LSTMAttention(50,2).to(device)

y = sa(batch[0])
print("Taille des sorties du classifieur :", y[0].shape, "\net de la sortie des attentions :", y[1].shape)

# ///////////////////////////////////////////////////////////////////////////////////////////////// </quick testing> ////

Taille des sorties du classifieur : torch.Size([64, 2]) 
et de la sortie des attentions : torch.Size([754, 64, 1])


### Baseline

In [5]:
trainAttention()


//////////////////// Attention-based LinNet : naive baseline /////////////////




LOSS: 		train 0.692788302898407 		test 0.6871360540390015
ACCURACY: 	train 0.5276041666666667 		test 0.616426282051282

LOSS: 		train 0.6834448575973511 		test 0.6797183752059937
ACCURACY: 	train 0.6122996794871794 		test 0.6495592948717949

LOSS: 		train 0.6762170195579529 		test 0.6731992363929749
ACCURACY: 	train 0.641426282051282 		test 0.6560496794871795

LOSS: 		train 0.6698512434959412 		test 0.6676980257034302
ACCURACY: 	train 0.6553685897435897 		test 0.6552483974358975

LOSS: 		train 0.6641855835914612 		test 0.6623219847679138
ACCURACY: 	train 0.6614182692307692 		test 0.6623798076923076

LOSS: 		train 0.6591483950614929 		test 0.6577063202857971
ACCURACY: 	train 0.6636618589743589 		test 0.6639022435897436

LOSS: 		train 0.6546140909194946 		test 0.6534661650657654
ACCURACY: 	train 0.666786858974359 		test 0.6619391025641026

LOSS: 		train 0.6503522396087646 		test 0.6496816873550415
ACCURACY: 	train 0.6706730769230769 		test 0.6642227564102564

LOSS: 		train 0.64654767513


LOSS: 		train 0.5681400299072266 		test 0.5721026659011841
ACCURACY: 	train 0.721073717948718 		test 0.7170673076923076

LOSS: 		train 0.567707359790802 		test 0.5719293355941772
ACCURACY: 	train 0.7226362179487179 		test 0.7144230769230769

LOSS: 		train 0.5669922232627869 		test 0.5712614059448242
ACCURACY: 	train 0.7227564102564102 		test 0.7158253205128206

LOSS: 		train 0.566495418548584 		test 0.5703666806221008
ACCURACY: 	train 0.7221955128205129 		test 0.7187099358974359

LOSS: 		train 0.5659566521644592 		test 0.5705655217170715
ACCURACY: 	train 0.7243189102564103 		test 0.7157451923076923

LOSS: 		train 0.5652347207069397 		test 0.5703115463256836
ACCURACY: 	train 0.7229166666666667 		test 0.7153846153846154

LOSS: 		train 0.5650387406349182 		test 0.5692270994186401
ACCURACY: 	train 0.7241987179487179 		test 0.7191506410256411

LOSS: 		train 0.5644022226333618 		test 0.5686797499656677
ACCURACY: 	train 0.7241185897435898 		test 0.7185096153846153

LOSS: 		train 0.5637531280

### Attention simple

In [6]:
trainAttention(model=SimpleAttention)


///////////////// Attention-based LinNet : basic implementation //////////////




LOSS: 		train 0.6848136186599731 		test 0.6799988150596619
ACCURACY: 	train 0.5820913461538462 		test 0.6088541666666667

LOSS: 		train 0.674388587474823 		test 0.6707136034965515
ACCURACY: 	train 0.6256410256410256 		test 0.624238782051282

LOSS: 		train 0.6640130877494812 		test 0.6593021154403687
ACCURACY: 	train 0.654727564102564 		test 0.657051282051282

LOSS: 		train 0.6511582732200623 		test 0.6449893116950989
ACCURACY: 	train 0.6752003205128205 		test 0.6809294871794872

LOSS: 		train 0.6339248418807983 		test 0.6253759860992432
ACCURACY: 	train 0.6959134615384616 		test 0.7028445512820513

LOSS: 		train 0.6106662154197693 		test 0.6004735231399536
ACCURACY: 	train 0.7204727564102564 		test 0.7235576923076923

LOSS: 		train 0.585597813129425 		test 0.5776975750923157
ACCURACY: 	train 0.7377804487179487 		test 0.7348557692307692

LOSS: 		train 0.5637266039848328 		test 0.5582751035690308
ACCURACY: 	train 0.7471955128205128 		test 0.7425080128205128

LOSS: 		train 0.545373916625


LOSS: 		train 0.4504975378513336 		test 0.4498746991157532
ACCURACY: 	train 0.7920272435897436 		test 0.7936298076923077

LOSS: 		train 0.4501482844352722 		test 0.4502839148044586
ACCURACY: 	train 0.7922275641025641 		test 0.7926282051282051

LOSS: 		train 0.4498780071735382 		test 0.44962745904922485
ACCURACY: 	train 0.7921073717948718 		test 0.79375

LOSS: 		train 0.44984009861946106 		test 0.44942474365234375
ACCURACY: 	train 0.7927884615384615 		test 0.7935096153846154

LOSS: 		train 0.4495348036289215 		test 0.4493943154811859
ACCURACY: 	train 0.7935897435897435 		test 0.7934695512820513

LOSS: 		train 0.4490576982498169 		test 0.4497293531894684
ACCURACY: 	train 0.7931089743589743 		test 0.7925480769230769

LOSS: 		train 0.44927147030830383 		test 0.4488200545310974
ACCURACY: 	train 0.7934294871794871 		test 0.7937900641025641

LOSS: 		train 0.44901642203330994 		test 0.4489074945449829
ACCURACY: 	train 0.7931089743589743 		test 0.7934695512820513

LOSS: 		train 0.4487987160682

### Attention avec questions et valeurs

In [3]:
trainAttention(model=FurtherAttention)


///////////////// Attention-based LinNet : further improvements //////////////




LOSS: 		train 0.5530955195426941 		test 0.5044490098953247
ACCURACY: 	train 0.7233573717948718 		test 0.7489583333333333

LOSS: 		train 0.4843697249889374 		test 0.4750230312347412
ACCURACY: 	train 0.7717548076923076 		test 0.7758012820512821

LOSS: 		train 0.4782238006591797 		test 0.48473531007766724
ACCURACY: 	train 0.7761217948717949 		test 0.7670673076923077

LOSS: 		train 0.47265079617500305 		test 0.4806292653083801
ACCURACY: 	train 0.7812900641025641 		test 0.7775240384615385

LOSS: 		train 0.46334418654441833 		test 0.4727323651313782
ACCURACY: 	train 0.7850560897435898 		test 0.7781650641025641

LOSS: 		train 0.46811383962631226 		test 0.4747752547264099
ACCURACY: 	train 0.7808092948717948 		test 0.7776842948717949

LOSS: 		train 0.4662545621395111 		test 0.4773558974266052
ACCURACY: 	train 0.7833333333333333 		test 0.7703125

LOSS: 		train 0.4624371826648712 		test 0.47015076875686646
ACCURACY: 	train 0.7857371794871795 		test 0.7809695512820513

LOSS: 		train 0.45788735151

### Ajout d'une LSTM

In [5]:
trainAttention(model=LSTMAttention)


//////////////////// Attention-based LinNet : adding an LSTM /////////////////

Restarting from previous state.





Done.



### Régularisation par l'entropie

In [5]:
trainAttention(model=FurtherAttention,reg=0.001)


///////////////// Attention-based LinNet : further improvements //////////////




LOSS: 		train 0.885619580745697 		test 0.6911413073539734
ACCURACY: 	train 0.5024839743589744 		test 0.5283253205128206

LOSS: 		train 0.7710341215133667 		test 0.6897268891334534
ACCURACY: 	train 0.5338141025641026 		test 0.5344951923076923

LOSS: 		train 0.752338707447052 		test 0.689031720161438
ACCURACY: 	train 0.5372996794871795 		test 0.5374198717948718

LOSS: 		train 0.7437063455581665 		test 0.6859720349311829
ACCURACY: 	train 0.541826923076923 		test 0.5448317307692307

LOSS: 		train 0.7383107542991638 		test 0.684909999370575
ACCURACY: 	train 0.5443509615384615 		test 0.5449919871794872

LOSS: 		train 0.7343741059303284 		test 0.6844171285629272
ACCURACY: 	train 0.5464743589743589 		test 0.5498798076923077

LOSS: 		train 0.7312635779380798 		test 0.6835806369781494
ACCURACY: 	train 0.5467147435897436 		test 0.5455128205128205

LOSS: 		train 0.728567361831665 		test 0.6830191612243652
ACCURACY: 	train 0.5493990384615385 		test 0.5501602564102565

LOSS: 		train 0.7263528108596


LOSS: 		train 0.6647258996963501 		test 0.6416763067245483
ACCURACY: 	train 0.6348157051282052 		test 0.6357371794871794

LOSS: 		train 0.6639215350151062 		test 0.6405137777328491
ACCURACY: 	train 0.6364583333333333 		test 0.6366586538461538

LOSS: 		train 0.662814736366272 		test 0.6397205591201782
ACCURACY: 	train 0.6377003205128206 		test 0.639102564102564

LOSS: 		train 0.6618578433990479 		test 0.6389337778091431
ACCURACY: 	train 0.6395432692307692 		test 0.6387419871794872

LOSS: 		train 0.6607645153999329 		test 0.6377782225608826
ACCURACY: 	train 0.6397836538461539 		test 0.6408253205128205

LOSS: 		train 0.660081148147583 		test 0.6370208263397217
ACCURACY: 	train 0.6431490384615385 		test 0.6434294871794872

LOSS: 		train 0.6589116454124451 		test 0.6363098621368408
ACCURACY: 	train 0.6433092948717949 		test 0.6441105769230769

LOSS: 		train 0.6580570340156555 		test 0.6345755457878113
ACCURACY: 	train 0.6451923076923077 		test 0.6473958333333333

LOSS: 		train 0.6568768024

### Visualisation des attentions

In [4]:
savepath = Path("attentionfurther.pch")
if savepath.is_file():
    with savepath.open("rb") as fp :
        state = torch.load(fp)
index = 2000

def get_attention_words_for_example(index_example,model,folder=foldertext_test,nb_words=10):
    example = folder[index_example]
    s = " ".join([id2word[i] for i in example[0]])
    embedded_example = padding([example])
    print("The example is ",s)
    print("********************************")
    preds, attentions = model(embedded_example[0])
    indices_firsts = [torch.argsort(attentions.squeeze(1).squeeze(1)).tolist().index(i) for i in range(nb_words)]
    words = [id2word[example[0][i]] for i in indices_firsts]
    print("The ten words with the most attention are :",words)
get_attention_words_for_example(index,state.model)

The example is  this is a __OOV__ biography of a star of the black and white minstrel __OOV__ a certain dixie boy __OOV__ whether this person ever really existed i __OOV__ __OOV__ but considering the cast lists a certain __OOV__ lasses __OOV__ and roscoe karns playing said character as __OOV__ i assume the man did exist and that this is a white-washed __OOV__ the __OOV__ version of his __OOV__ the __OOV__ such as it __OOV__ follows dixie boy from career heights to depression at the death of his wife in __OOV__ his abandonment of the child to __OOV__ and his return at his __OOV__ sixteenth birthday and stage debut for __OOV__ another forgotten __OOV__ benny __OOV__ plays dixie __OOV__ the man has a lovely baritone voice but no acting talent whatsoever and is a boring lump on __OOV__ gladys george valiantly tries her best to enliven the works to no __OOV__ judy clark does the best impersonation of betty hutton __OOV__ ever seen although i believe she thought she was being __OOV__ the sco